#### Clasificación de dígitos en tiempo real

Este programa utiliza la cámara web para capturar dígitos y clasificarlos con el modelo entrenado `final_model.h5`.  

- La detección de la región de interés (ROI) se puede activar automáticamente con la tecla A.  
- Presionando Q se cierra el programa.  

El modelo muestra en tiempo real el dígito detectado y su nivel de confianza, permitiendo una clasificación rápida y visual.

In [17]:
import cv2
import numpy as np
import tensorflow as tf

# =========================
# Cargar modelo
# =========================
model = tf.keras.models.load_model("final_model.h5")

# =========================
# Configuración cámara
# =========================
cap = cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)

# =========================
# Variables de ROI
# =========================
roi_selected = False
roi_start = (0, 0)
roi_end = (0, 0)
auto_detection = True

# =========================
# Función: Auto-detección de dígitos
# =========================
def auto_detect_digits(frame):
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    blurred = cv2.GaussianBlur(gray, (7, 7), 0)
    thresh = cv2.adaptiveThreshold(
        blurred, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, 
        cv2.THRESH_BINARY_INV, 11, 2
    )
    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    digit_contours = []
    for contour in contours:
        x, y, w, h = cv2.boundingRect(contour)
        area = cv2.contourArea(contour)
        if 500 < area < 40000 and 20 < w < 200 and 30 < h < 200:
            aspect_ratio = w / h
            if 0.2 < aspect_ratio < 1.5:
                digit_contours.append((x, y, w, h, area))

    if digit_contours:
        digit_contours.sort(key=lambda x: x[4], reverse=True)
        x, y, w, h, _ = digit_contours[0]
        margin = 15
        x = max(0, x - margin)
        y = max(0, y - margin)
        w = min(frame.shape[1] - x, w + 2 * margin)
        h = min(frame.shape[0] - y, h + 2 * margin)
        return (x, y, w, h)
    return None

# =========================
# Loop principal
# =========================
try:
    while True:
        ret, frame = cap.read()
        if not ret:
            break

        frame_with_roi = frame.copy()
        prob_display = np.ones((480, 400, 3), dtype=np.uint8) * 255

        # Auto-detección
        if auto_detection:
            auto_roi = auto_detect_digits(frame)
            if auto_roi:
                x, y, w, h = auto_roi
                roi_start = (x, y)
                roi_end = (x + w, y + h)
                roi_selected = True

        # Dibujar ROI
        if roi_selected:
            cv2.rectangle(frame_with_roi, roi_start, roi_end, (255, 0, 0), 2)
            x1, y1 = roi_start
            x2, y2 = roi_end
            roi = frame[y1:y2, x1:x2]
            if roi.size > 0:
                roi_gray = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)
                resized = cv2.resize(roi_gray, (28,28))
                normalized = 1 - (resized / 255.0)
                input_img = normalized.reshape(1,28,28,1)

                # Predicción
                prediction = model.predict(input_img, verbose=0)
                predicted_label = np.argmax(prediction)
                max_prob = np.max(prediction)

                # Mostrar predicción en frame
                cv2.putText(frame_with_roi, f'Pred: {predicted_label} Conf: {max_prob:.2f}',
                            (x1, max(y1-10,20)), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0,255,0), 2)

                # Mostrar probabilidades en display lateral
                cv2.putText(prob_display, 'PROBABILIDADES:', (20,30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0,0,0), 2)
                for i, prob in enumerate(prediction[0]):
                    y_pos = 70 + i*35
                    color = (0,255,0) if i==predicted_label else (0,0,255)
                    bar_width = int(prob*200)
                    cv2.rectangle(prob_display, (150, y_pos-20), (150+bar_width, y_pos-5), color, -1)
                    cv2.putText(prob_display, f'{i}: {prob:.3f}', (20, y_pos), cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)

        # Mostrar imagen combinada
        combined = np.hstack((frame_with_roi, prob_display))
        cv2.imshow('Clasificacion en Tiempo Real', combined)
        cv2.moveWindow('Clasificacion en Tiempo Real', 100, 100)

        # Controles
        key = cv2.waitKey(1) & 0xFF
        if key == ord('q'):
            break
        elif key == ord('a'):
            auto_detection = True
            roi_selected = False

except KeyboardInterrupt:
    print("Interrupcion por teclado")
finally:
    cap.release()
    cv2.destroyAllWindows()